In [ ]:
# Following is the implementation without using ordereddict.
class Node:
    def __init__(self, key, val):
        self.val = val
        self.key = key
        self.next = None
        self.prev = None
        
class LinkedList:
    def __init__(self):
        self.head = None
        self.tail = None
    
    def insert(self, node):
        if self.head is None:
            self.head = node
        else:
            self.tail.next = node
            node.prev = self.tail
        self.tail = node
            
    def delete(self, node):
        if node.prev:
            node.prev.next = node.next
        else:
            self.head = node.next
            
        if node.next:
            node.next.prev = node.prev
        else:
            self.tail = node.prev

class LRUCache:
    def __init__(self, capacity):
        self.list = LinkedList()
        self.dict = {}
        self.capacity = capacity
        
    def _insert(self, key, val):
        node = Node(key, val)
        self.list.insert(node)
        self.dict[key] = node

    def get(self, key):
        if key in self.dict:
            val = self.dict[key].val
            self.list.delete(self.dict[key])
            self._insert(key, val)
            return val
        return -1

    def set(self, key, val):
        if key in self.dict:
            self.list.delete(self.dict[key])
        elif len(self.dict) == self.capacity:
            del self.dict[self.list.head.key]
            self.list.delete(self.list.head)
        self._insert(key, val)

lRUCache =  LRUCache(2)
lRUCache.set(1, 1) # cache is {1=1}
lRUCache.set(2, 2) # cache is {1=1, 2=2}

print(lRUCache.get(1))    # return 1
lRUCache.set(3, 3) # LRU key was 2, evicts key 2, cache is {1=1, 3=3}
print(lRUCache.get(2) )   # returns -1 (not found)
lRUCache.set(4, 4) # LRU key was 1, evicts key 1, cache is {4=4, 3=3}
print(lRUCache.get(1))   # return -1 (not found)
print(lRUCache.get(3))    # return 3
print(lRUCache.get(4))   # return 4

In [ ]:
class TrieNode(object):
    def __init__(self):
        self.is_file = False
        self.children = {}
        self.content = ""

class FileSystem(object):

    def __init__(self):
        self.root = TrieNode()


    def ls(self, path):
        """
        :type path: str
        :rtype: List[str]
        """
        curr = self.__getNode(path)

        if curr.is_file:
            return [self.__split(path, '/')[-1]]

        return sorted(curr.children.keys())


    def mkdir(self, path):
        """
        :type path: str
        :rtype: void
        """
        curr = self.__putNode(path)
        curr.is_file = False


    def addContentToFile(self, filePath, content):
        """
        :type filePath: str
        :type content: str
        :rtype: void
        """
        curr = self.__putNode(filePath)
        curr.is_file = True
        curr.content += content


    def readContentFromFile(self, filePath):
        """
        :type filePath: str
        :rtype: str
        """
        return self.__getNode(filePath).content

    def __getNode(self, path):
        curr = self.root
        for s in self.__split(path, '/'):
            curr = curr.children[s]
        return curr


    def __putNode(self, path):
        curr = self.root
        for s in self.__split(path, '/'):
            if s not in curr.children:
                curr.children[s] = TrieNode()
            curr = curr.children[s]
        return curr


    def __split(self, path, delim):
        if path == '/':
            return []
        return path.split('/')[1:]

# Your FileSystem object will be instantiated and called as such:
fs = FileSystem()
param_1 = fs.ls("/")
fs.mkdir("/a/b/c")
fs.addContentToFile("/a/b/c/d","hello")
fs.ls("/")
fs.readContentFromFile("/a/b/c/d")

In [ ]:
class Node:
    def __init__(self, value):
        self.value = value
        self.children = {} # route : Node
    
class FileSystem:
    def __init__(self):
        self.root = Node(None)

    def createPath(self, path: str, value: int) -> bool:        
        # Be careful! Python string split contains empty strings!
        paths = path.split('/')
        
        n = self.root
        for i, s in enumerate(paths):
            if not s:
                continue
            if s in n.children:
                if i == len(paths) - 1:
                    # path already existed
                    return False
                else:
                    n = n.children[s]
            else:
                if i == len(paths) - 1:
                    n.children[s] = Node(value)
                else:
                    # parent path does not exist
                    return False
        return True
            
    def get(self, path: str) -> int:
        n = self.root
        paths = path.split('/')
        for i, s in enumerate(paths):
            if not s:
                continue
                
            if s in n.children:
                n = n.children[s]
            else:
                # path does not exist
                return -1
            
        return n.value


fileSystem =  FileSystem()

fileSystem.createPath("/leet", 1) # return true
fileSystem.createPath("/leet/code", 2) # return true
fileSystem.get("/leet/code") # return 2
fileSystem.createPath("/c/d", 1) # return false because the parent path "/c" doesn't exist.
fileSystem.get("/c")# return -1 because this path doesn't exist.
